# Ciencia de Datos Acelerado con GPU: Introducción a BlazingSQL y RAPIDS

Programar sobre GPUs puede ser intimidante al inicio. En el pasado, necesitabas un sólido conocimiento en lenguajes como C++ y CUDA, así como la capacidad de pensar *en paralelo*. Hoy, con [RAPIDS](https://rapids.ai) y [BlazingSQL](https://blazingsql.com), puedes comenzar a usar el inmenso poder de la GPU de forma inmediata. Todo esto, con el mínimo de cambios en código: ya sea usando herramientas del ecosistema PyData como *pandas* o *Scikit-Learn*, o te resulte más familiar *SQL*, RAPIDS y BlazingSQL te permiten lograr aceleraciones increíbles usando la tecnología de una GPU.

## Imports
Primero, importemos las librerías que vamos a necesitar.

In [ ]:
import cudf
import blazingsql as bsql
import s3fs
import numpy as np
from collections import OrderedDict

### `BlazingContext`
Debes establecer un `BlazingContext` para conectarte a una instancia BlazingSQL y con ello poder crear tablas, ejecutar consultas y básicamente hacer cualquier cosa con tu data usando BlazingSQL.

In [ ]:
bc = bsql.BlazingContext()

El `BlazingContext` es un punto de entrada para todo el abanico de posibilidades que ofrece BlazingSQL. En esta instancia particular iniciamos el `BlazingContext` con parámetros en default, pero existen muchas formas de customizar y lograr mayores capacidades.

|Argument|Required|Description|Defaults|
|:-------|:------:|:----------|-------:|
allocator|      No|Las opciones son: "default", "managed". Cuando figura "managed" usa Unified Virtual Memory (UVM) y  puede usar memoria del sistema si la memoria del GPU se agota, o "existing" cuando asumimos que ya está configurado el rmm allocator y por ello no se inicializa (esto para usuarios avanzados).|"managed"
dask_client|No|El cliente Dask es usado para comunicación con otros nodos. Esto es sólo necesario para ejecutar BlazingSQL con múltiples nodos.|None
enable_logging|No|Si figura en True, el memory allocator logging estará activo, pero puede impactar de forma negativa en la performance. Esto es para usuarios avanzados.|False
initial_pool_size|No|Tamaño inicial de memory pool en bytes (si pool=True). Si no, estará en default para usar la mitad de la memoria de GPU.|None
pool|No|Si es True, If True, se asigna la memory pool en el inicio. Esto puede mejorar considerablemente el performance.|False
network_interface|No|Interface de red usada para comunicarse con el dask-scheduler. Mira la nota debajo.|'eth0'
config_options|No|Un diccionario para configurar ciertos parámetros en el motor.|

## Leyendo data y ejecutando consultas

Hay dos maneras para cargar y ejecutar data usando las herramientas del ecosistema de RAPIDS: cargar directamente a la memoria usando `cudf` o  usando `.create_table()` del  `BlazingContext`.

### Data de vuelos

In [ ]:
flight_data_path = 's3://bsql/data/air_transport/flight_ontime_2020-0[1-5].parquet'
s3 = s3fs.S3FileSystem(anon=True)
files = [f's3://{f}' for f in s3.glob(flight_data_path)]
files

#### cuDF

In [ ]:
%%time
flights = []

for f in files:
    flights.append(cudf.read_parquet(f, storage_options={'anon': True}))
    
flights = cudf.concat(flights)

In [ ]:
flights.head(5)

In [ ]:
print(f'Número total de vuelos en el dataset: {len(flights):,}')

#### BlazingSQL

In [ ]:
bc.s3('bsql', bucket_name = 'bsql')

In [ ]:
bc.create_table('air_transport', files)

In [ ]:
%%time
bc.sql('SELECT * FROM air_transport LIMIT 5')

In [ ]:
print(f'Número total de vuelos en el dataset: {bc.sql("SELECT COUNT(*) AS CNT FROM air_transport")["CNT"].iloc[0]:,}')

#### Columnas y tipos de data

In [ ]:
flights.columns

In [ ]:
flights.dtypes

El `BlazingContext` retorna un objeto cuDF DataFrame, por lo que tenemos acceso al mismo API.

In [ ]:
bc_df = bc.sql('SELECT * FROM air_transport LIMIT 5')
type(bc_df)

In [ ]:
bc_df.columns

In [ ]:
bc_df.dtypes

### Data de vuelos y aeropuertos

In [ ]:
airports_path = 's3://bsql/data/air_transport/airports.csv'
airlines_path = 's3://bsql/data/air_transport/airlines.csv'

In [ ]:
airports_dtypes = OrderedDict([
      ('Airport ID', 'int64')
    , ('Name', 'str')
    , ('City', 'str')
    , ('Country', 'str')
    , ('IATA', 'str')
    , ('ICAO', 'str')
    , ('Latitude', 'float64')
    , ('Longitude', 'float64')
    , ('Altitude', 'int64')
    , ('Timezone', 'str')
    , ('DST', 'str')
    , ('Type', 'str')
    , ('Source', 'str')
])

airports = cudf.read_csv(
    airports_path
    , names=list(airports_dtypes.keys())
    , dtype=list(airports_dtypes.values())
    , storage_options={'anon': True}
)
airports.head()

In [ ]:
airlines_dtypes = OrderedDict([
    ('Airline ID', 'int64')
    , ('Name', 'str')
    , ('Alias', 'str')
    , ('IATA', 'str')
    , ('ICAO', 'str')
    , ('Callsign', 'str')
    , ('Country', 'str')
    , ('Active', 'str')
])

airlines = cudf.read_csv(
    airlines_path
    , names=list(airlines_dtypes.keys())
    , dtype=list(airlines_dtypes.values())
    , storage_options={'anon': True}
)
airlines.head()

Puedes crear tablas BlazingSQL directamente desde cuDF DataFrames.

In [ ]:
bc.create_table('airports', airports)
bc.create_table('airlines', airlines)

Y ahora, podemos consultar y unir estos datasets.

In [ ]:
%%time
bc.sql('''
    SELECT A.FL_DATE
        , A.OP_UNIQUE_CARRIER
        , B.Name AS CARRIER_NAME
        , A.ORIGIN
        , C.Name AS ORIGIN_NAME
        , C.City AS ORIGIN_CITY
        , A.DEST
        , D.Name AS DEST_NAME
        , D.City AS DEST_CITY
    FROM air_transport AS A
    LEFT OUTER JOIN airlines AS B
        ON A.OP_UNIQUE_CARRIER = B.IATA
    LEFT OUTER JOIN airports AS C
        ON A.ORIGIN = C.IATA
    LEFT OUTER JOIN airports AS D
        ON A.DEST = D.IATA
    LIMIT 4
''')

Lo hermoso de este ecosistema, y particualrmente de BlazingSQL, es la inter-operatividad con RAPIDS: podemos crear tablas desde cudf y cualquier formato soportado por cuDF, ya sea local o remoto; podemos registrar buckets desde `s3`, `gcp` con el `BlazingContext` y con soporte para Azure en futuros releases. Tambien podemos crear de forma sencilla, tablas directamente desde archivos y escribir código que retorne un cuDF DataFrame uniendo Parquet y archivos CSV en sólo un par de líneas!

In [ ]:
bc.create_table('airports_table', airports_path, names=list(airports_dtypes.keys()), dtype=list(airports_dtypes.values()))
bc.create_table('airlines_table', airlines_path, names=list(airlines_dtypes.keys()), dtype=list(airlines_dtypes.values()))

In [ ]:
%%time
bc.sql('''
    SELECT A.FL_DATE
        , A.OP_UNIQUE_CARRIER
        , B.Name AS CARRIER_NAME
        , A.ORIGIN
        , C.Name AS ORIGIN_NAME
        , C.City AS ORIGIN_CITY
        , A.DEST
        , D.Name AS DEST_NAME
        , D.City AS DEST_CITY
    FROM air_transport AS A                // READING FROM PARQUET
    LEFT OUTER JOIN airlines AS B
        ON A.OP_UNIQUE_CARRIER = B.IATA
    LEFT OUTER JOIN airports_table AS C    // READING FROM CSV
        ON A.ORIGIN = C.IATA
    LEFT OUTER JOIN airports_table AS D    // READING FROM CSV
        ON A.DEST = D.IATA
    LIMIT 4
''')

In [ ]:
%%time
(
    flights[['FL_DATE', 'OP_UNIQUE_CARRIER', 'ORIGIN', 'DEST']]
    .merge(airlines[['IATA', 'Name']], left_on='OP_UNIQUE_CARRIER', right_on='IATA')
    .rename(columns={'Name': 'CARRIER_NAME'})
    .drop(columns=['IATA'])
    .merge(airports[['IATA', 'Name', 'City']], left_on='ORIGIN', right_on='IATA')
    .rename(columns={'Name': 'ORIGIN_NAME', 'City': 'ORIGIN_CITY'})
    .drop(columns=['IATA'])
    .merge(airports[['IATA', 'Name', 'City']], left_on='DEST', right_on='IATA')
    .rename(columns={'Name': 'DEST_NAME', 'City': 'DEST_CITY'})
    .drop(columns=['IATA'])
).head()

## Preguntas

### 1. ¿Cuántos aeropuertos hay en el dataset?

In [ ]:
print(f'Existen {len(flights["ORIGIN"].unique())} aeropuertos en el dataset')

In [ ]:
print(f'Existen {bc.sql("SELECT COUNT(DISTINCT ORIGIN) AS CNT FROM air_transport")["CNT"][0]} aeropuertos en el dataset')

### 2. ¿Cuántos vuelos tuvieron retraso y cuántos partieron a tiempo? ¿Cuál es la distribución?

In [ ]:
print(f'{len(flights[flights["DEP_DELAY"] > 0]):,} vuelos con retraso y {len(flights[flights["DEP_DELAY"] <= 0]):,} vuelos a tiempo')

In [ ]:
### calculando la distribución
n_bins = 100

delays = flights[flights['DEP_DELAY'] >  0]['DEP_DELAY']
ontime = flights[flights['DEP_DELAY'] <= 0]['DEP_DELAY']

In [ ]:
%%time
del_bins = np.array([i * 15 for i in range(0, n_bins)], dtype='float64')
delays_binned = delays.digitize(del_bins)
delays_histogram = delays_binned.groupby().count() / len(delays)
(
    delays_histogram
    .set_index(del_bins[delays_histogram.index.to_array()-1])
    .to_pandas()
    .plot(kind='bar', figsize=(20,9), ylim=[0,1.0], title='Distribución de salidas retrasadas')
)

In [ ]:
%%time
ontime_bins = np.array([i * (-1) for i in range(n_bins,0,-1)], dtype='float64')
ontime_binned = ontime.digitize(ontime_bins)
ontime_histogram = ontime_binned.groupby().count() / len(ontime)
(
    ontime_histogram
    .set_index(ontime_bins[ontime_histogram.index.to_array()-1])
    .to_pandas()
    .plot(kind='bar', figsize=(20,9), ylim=[0,1.0], title='Distribución de salidas a tiempo')
)

### 3. ¿Cuáles son las 10 aerolíneas top y aeropuertos con mayores retrasos en por lo menos 1000 vuelos? ¿Cuál es el promedio de demora?

In [ ]:
delays = flights[flights['DEP_DELAY'] >  0][['DEP_DELAY', 'ORIGIN', 'DEST', 'OP_UNIQUE_CARRIER']]
ontime = flights[flights['DEP_DELAY'] <= 0][['DEP_DELAY', 'ORIGIN', 'DEST', 'OP_UNIQUE_CARRIER']]

In [ ]:
bc.create_table('delays', delays)
bc.create_table('ontime', ontime)

#### Los que presentaron mayores retrasos

In [ ]:
%%time
bc.sql('''
    SELECT A.ORIGIN
        , B.Name AS ORIGIN_Airport
        , B.City AS ORIGIN_City
        , B.Country AS ORIGIN_Country
        , COUNT(*) AS DELAY_CNT
        , AVG(DEP_DELAY) AS AVG_DELAY
    FROM delays AS A
    LEFT OUTER JOIN airports AS B
        ON A.ORIGIN = B.IATA
    GROUP BY A.ORIGIN
        , B.Name
        , B.City
        , B.Country
    HAVING COUNT(*) > 1000
    ORDER BY AVG(DEP_DELAY) DESC
    LIMIT 10
''')

In [ ]:
%%time
bc.sql('''
    SELECT A.DEST
        , B.Name AS DEST_Airport
        , B.City AS DEST_City
        , B.Country AS DEST_Country
        , COUNT(*) AS DELAY_CNT
        , AVG(DEP_DELAY) AS AVG_DELAY
    FROM delays AS A
    LEFT OUTER JOIN airports AS B
        ON A.DEST = B.IATA
    GROUP BY A.DEST
        , B.Name
        , B.City
        , B.Country
    HAVING COUNT(*) > 1000
    ORDER BY AVG(DEP_DELAY) DESC
    LIMIT 10
''')

In [ ]:
%%time
bc.sql('''
    SELECT A.OP_UNIQUE_CARRIER AS CARRIER
        , B.Name AS CARRIER_Name
        , B.Country AS CARRIER_Country
        , COUNT(*) AS DELAY_CNT
        , AVG(DEP_DELAY) AS AVG_DELAY
    FROM delays AS A
    LEFT OUTER JOIN airlines AS B
        ON A.OP_UNIQUE_CARRIER = B.IATA
    GROUP BY A.OP_UNIQUE_CARRIER
        , B.Name
        , B.Country
    HAVING COUNT(*) > 1000
    ORDER BY AVG(DEP_DELAY) DESC
    LIMIT 10
''')

#### Los más puntuales

In [ ]:
%%time
bc.sql('''
    SELECT A.ORIGIN
        , B.Name AS ORIGIN_Airport
        , B.City AS ORIGIN_City
        , B.Country AS ORIGIN_Country
        , COUNT(*) AS ONTIME_CNT
        , AVG(DEP_DELAY) AS AVG_ONTIME
    FROM ontime AS A
    LEFT OUTER JOIN airports AS B
        ON A.ORIGIN = B.IATA
    GROUP BY A.ORIGIN
        , B.Name
        , B.City
        , B.Country
    HAVING COUNT(*) > 1000
    ORDER BY AVG(DEP_DELAY) DESC
    LIMIT 10
''')

In [ ]:
%%time
bc.sql('''
    SELECT A.DEST
        , B.Name AS DEST_Airport
        , B.City AS DEST_City
        , B.Country AS DEST_Country
        , COUNT(*) AS ONTIME_CNT
        , AVG(DEP_DELAY) AS AVG_ONTIME
    FROM ontime AS A
    LEFT OUTER JOIN airports AS B
        ON A.DEST = B.IATA
    GROUP BY A.DEST
        , B.Name
        , B.City
        , B.Country
    HAVING COUNT(*) > 1000
    ORDER BY AVG(DEP_DELAY) DESC
    LIMIT 10
''')

In [ ]:
%%time
bc.sql('''
    SELECT A.OP_UNIQUE_CARRIER AS CARRIER
        , B.Name AS CARRIER_Name
        , B.Country AS CARRIER_Country
        , AVG(DEP_DELAY) AS AVG_ONTIME
    FROM ontime AS A
    LEFT OUTER JOIN airlines AS B
        ON A.OP_UNIQUE_CARRIER = B.IATA
    GROUP BY A.OP_UNIQUE_CARRIER
        , B.Name
        , B.Country
    HAVING COUNT(*) > 1000
    ORDER BY AVG(DEP_DELAY) DESC
    LIMIT 10
''')

## Fechas, strings, oh my...

Un error común es creer que usar GPUs es sólo útil para cálculos numéricos. Sin embargo, con RAPIDS y BlazingSQL puedes realizar operaciones en fechas y strings fácilmente y a la velocidad de GPUs!

### Vuelos por mes y día de la semana

Aunque ya contamos con columnas como `YEAR` o `MONTH`, vamos a calcular estos valores nosotros mismos!

In [ ]:
%%time
flights['FL_DATE'] = flights['FL_DATE'].astype('datetime64[ms]')
dated = flights[['FL_DATE', 'OP_UNIQUE_CARRIER']]
dated['YEAR'] = dated['FL_DATE'].dt.year
dated['MONTH'] = dated['FL_DATE'].dt.month
dated['DAY'] = dated['FL_DATE'].dt.day
dated['DOW'] = dated['FL_DATE'].dt.dayofweek

In [ ]:
%%time
(
    dated
    .groupby(['YEAR','MONTH'])
    .agg({'FL_DATE': 'count'})
    .to_pandas()
    .plot(kind='bar', figsize=(12,9), title='Vuelos totales por mes')
)

In [ ]:
%%time
(
    dated
    .groupby(['MONTH','DAY', 'DOW'])
    .agg({'FL_DATE': 'count'})
    .reset_index()
    .groupby(['DOW'])
    .agg({'FL_DATE': 'mean'})
    .to_pandas()
    .plot(kind='bar', figsize=(12,9), title='Numero promedio de vuelos por dia de semana')
)